In [1]:
# Pytorch model to do classification. Dataset from folder

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [3]:
# Use pretrained model (Resnet)
model = torchvision.models.resnet18(pretrained=True)

/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Input RGB image, normalise to resnet18 and all those stuff
transform = transforms.Compose(
    [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])])

# Load dataset from folder
dataset = torchvision.datasets.ImageFolder(root='/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/augmented_images/', transform=transform)

In [5]:
# Random split dataset into train and test
trainset, testset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])


In [6]:
# Dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                            shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                            shuffle=False, num_workers=2)


In [7]:
# Freeze all layers except last fc layer
for param in model.parameters():
    param.requires_grad = False
    
# Change last fc layer to 7 classes
model.fc = nn.Linear(512, 7)

In [8]:
# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
# Train, test in each epoch with tqdm progress bar, use functions for train and test
from tqdm import tqdm

def train():
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print("Training loss: ", running_loss/len(trainloader))

def test():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader, 0)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print("Testing loss: ", running_loss/len(testloader))
    print("Accuracy: ", correct/total)


In [11]:
# Train and test for 10 epochs
for epoch in range(10):
    print("Epoch: ", epoch)
    train()
    test()
    print("\n")



Epoch:  0


525it [00:36, 14.43it/s]

Training loss:  1.559882934207008



132it [00:09, 14.30it/s]

Testing loss:  1.34099572355097
Accuracy:  0.5035714285714286


Epoch:  1


...

Testing loss:  1.242415237607378
Accuracy:  0.5366666666666666


Epoch:  9



525it [00:37, 13.96it/s]

Training loss:  1.2175604204904464



132it [00:10, 13.01it/s]


Testing loss:  1.23200337498477
Accuracy:  0.5357142857142857




In [12]:
# Unfreeze all layers, and fine tune with smaller learning rate
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


In [13]:
# Train and test for 10 epochs
for epoch in range(20):
    print("Epoch: ", epoch)
    train()
    test()
    print("\n")


Epoch:  0


525it [00:44, 11.80it/s]

Training loss:  1.0733700163023812



132it [00:09, 14.08it/s]


Testing loss:  1.0321487992098837
Accuracy:  0.6076190476190476


Epoch:  1


...

Testing loss:  0.7754719113987504
Accuracy:  0.710952380952381


Epoch:  9



525it [00:45, 11.59it/s]

Training loss:  0.47424104304540726



132it [00:10, 12.99it/s]

Testing loss:  0.7742289404074351
Accuracy:  0.7088095238095238




In [16]:
# Make predictions with this model, on test directory, and epxort to csv as per kaggle format
import pandas as pd
import os
from PIL import Image

# Load test directory
test_dir = '/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/test'

# Submission df structure - file_name,label - Lable is predicted class (0-6)
submission_df = pd.DataFrame(columns=['file_name', 'label'])


# Iterate over all files in test directory
for file in tqdm(os.listdir(test_dir)):
    # Open image, resize, convert to tensor, normalise
    img = Image.open(os.path.join(test_dir, file))
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    
    # Get prediction
    output = model(img)
    _, predicted = torch.max(output.data, 1)
    
    # Add to submission df using pandas concat
    submission_df = pd.concat([submission_df, pd.DataFrame([[file, predicted.item()]], columns=['file_name', 'label'])], ignore_index=True)


100%|██████████| 5751/5751 [00:41<00:00, 140.01it/s]


In [17]:
# Export to csv
submission_df.to_csv('submission.csv', index=False)

In [18]:
# Save model
torch.save(model.state_dict(), 'model.pth')
